In [1]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [2]:
def combine_datasets(original_dir, augmented_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    classes = ['Monkeypox', 'Non_Monkeypox']
    
    for cls in classes:
        original_path = os.path.join(original_dir, cls)
        augmented_path = os.path.join(augmented_dir, f'{cls}_augmented')
        target_class_dir = os.path.join(target_dir, cls)
        
        if not os.path.exists(target_class_dir):
            os.makedirs(target_class_dir)
        
        # Copy original images
        for filename in os.listdir(original_path):
            source_file = os.path.join(original_path, filename)
            shutil.copy(source_file, target_class_dir)
        
        # Copy augmented images
        for filename in os.listdir(augmented_path):
            source_file = os.path.join(augmented_path, filename)
            shutil.copy(source_file, target_class_dir)

original_dir = 'dataset/Original Images'
augmented_dir = 'dataset/Augmented Images'
target_dir = 'dataset/combined_data'
combine_datasets(original_dir, augmented_dir, target_dir)


In [3]:
# Set up the ImageDataGenerator for data augmentation and rescaling
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)

# Load the training and validation data
train_generator = datagen.flow_from_directory(
    'dataset/combined_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'dataset/combined_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 2908 images belonging to 2 classes.
Found 512 images belonging to 2 classes.


In [4]:
# Load the base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Add new layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])




9406464/9406464 [==============================] - 1s 0us/step


In [5]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10


91/91 [==============================] - 66s 664ms/step - loss: 0.5396 - accuracy: 0.7132 - val_loss: 0.5103 - val_accuracy: 0.7617
Epoch 2/10
91/91 [==============================] - 37s 408ms/step - loss: 0.3613 - accuracy: 0.8518 - val_loss: 0.4652 - val_accuracy: 0.7910
Epoch 3/10
91/91 [==============================] - 37s 410ms/step - loss: 0.2947 - accuracy: 0.8876 - val_loss: 0.4709 - val_accuracy: 0.7754
Epoch 4/10
91/91 [==============================] - 40s 439ms/step - loss: 0.2511 - accuracy: 0.9099 - val_loss: 0.4508 - val_accuracy: 0.7754
Epoch 5/10
91/91 [==============================] - 38s 419ms/step - loss: 0.2227 - accuracy: 0.9209 - val_loss: 0.4714 - val_accuracy: 0.7695
Epoch 6/10
91/91 [==============================] - 37s 411ms/step - loss: 0.1931 - accuracy: 0.9367 - val_loss: 0.4556 - val_accuracy: 0.7793
Epoch 7/10
91/91 [==============================] - 36s 397ms/step - loss: 0.1761 - accuracy: 0.9426 - val_loss: 0.4633 - val_accuracy: 0.75

In [6]:
# Save the model
model.save('monkeypox_model.h5')


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
# (Optional) Load and evaluate the model
test_generator = datagen.flow_from_directory(
    'dataset/test_data',  # Assume you have a separate test dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load the saved model
loaded_model = tf.keras.models.load_model('monkeypox_model.h5')

# Evaluate the model
test_loss, test_acc = loaded_model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/test_data'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Step 1: Load the trained model
model = tf.keras.models.load_model('monkeypox_model.h5')

# Step 2: Prepare the image for prediction
def prepare_image(image_path, target_size=(224, 224)):
    # Load the image
    img = load_img(image_path, target_size=target_size)
    
    # Convert the image to a numpy array
    img_array = img_to_array(img)
    
    # Rescale the image (as done during model training)
    img_array = img_array / 255.0
    
    # Expand dimensions to match the input shape of the model (batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

# Step 3: Make a prediction on a new image
def predict_image(image_path):
    # Prepare the image
    img_array = prepare_image(image_path)
    
    # Make the prediction
    prediction = model.predict(img_array)
    
    # Convert prediction to class label
    if prediction[0] > 0.5:
        return "Monkeypox"
    else:
        return "Non Monkeypox"

# Step 4: Test with a random image
image_path = 'path_to_your_image.jpg'  # Replace with the path to your image
result = predict_image(image_path)
print(f'The image is predicted as: {result}')
